# Retail Demo Store Experimentation Workshop - Multi-Armed Bandit Experiment Exercise

In this exercise we will define and launch an experiment using a solution to the [multi-armed bandit problem](https://en.wikipedia.org/wiki/Multi-armed_bandit) to evaluate multiple recommendation approaches concurrently. If you have not already stepped through the **[3.1-Overview](./3.1-Overview.ipynb)** workshop notebook, please do so now as it provides the foundation built upon in this exercise. It is also suggested, but not required, to complete the **[3.2-AB-Experiment](./3.2-AB-Experiment.ipynb)** and **[3.3-Interleaving-Experiment](./3.3-Interleaving-Experiment.ipynb)** workshop notebooks.

Recommended Time: 30 minutes

## Prerequisites

Since this module uses the Retail Demo Store's Recommendation microservice to run experiments across variations that depend on the search and personalization features of the Retail Demo Store, it is assumed that you have either completed the [Search](../0-StartHere/Search.ipynb) and [Personalization](../1-Personalization/1.1-Personalize.ipynb) workshops or those resources have been pre-provisioned in your AWS environment. If you are unsure and attending an AWS managed event such as a workshop, check with your event lead.

## Exercise 3: Multi-Armed Bandit Experiment

In the first two exercises we demonstrated how to create and run experiments using a traditional A/B test and an interleaving recommendations test. Both of those approaches require the sequential steps of creating the experiment, running the experiment, and then evaluating the results to determine if a statistically valid preference emerged. The time when the experiment is running is typically referred to as the **exploration** phase since we are exposing users to two variations and gathering the data necessary to draw a conclusion. Only after the test has completed can we use the winning variation across all users to maximize conversion. This is referred to the **exploitation** phase. Minimizing exploration without jeopardizing the integrity of the results in order to maximize exploitation are fundamental elements to any successful experimentation strategy.

For this exercise, we will take a entirely different approach to evaluating multiple recommendation implementations concurrently where the best performing variation is measured and exploited in real-time based on user feedback but the other variations are still ocassionally explored should the probabilities for conversion change over time. This approach is often referred to as the multi-armed bandit problem since it is analagous to the gambler entering a room of slot machines (i.e. one-armed bandits) and having to decide which arm to pull, how many times to pull each arm, and when to try other machines to maximize the payout.

The multi-armed bandit approach is ideal for experimentation use-cases in short-lived and dynamic environments with many variations where longer drawn out testing approaches are unfeasible.

Although bandit testing can support tens of variations in a single experiment, we will use three variations for this exercise. The first variation will represent our current implementation using the **Default Product Resolver**, the second variation will use the **Similar Products Resolver**, and the third variation will use the **Personalize Recommendation Resolver**. We will simulate this experiment using the related products feature on the product detail page. The following screenshot illustrates what an active multi-armed bandit test would look like on the product detail page with experiment annotations.

![Multi-Armed Bandit](./images/ui-related-mab.png)

### MultiArmedBanditExperiment Class

Before stepping through creating and executing our multi-armed bandit test, let's look at the relevant source code for the **MultiArmedBanditExperiment** class that implements this experiment type in the Retail Demo Store project.

As noted in the **3.1-Overview** notebook, all experiment types are subclasses of the abstract **Experiment** class. See **[3.1-Overview](./3.1-Overview.ipynb)** for more details on the experimentation framework.

The `MultiArmedBanditExperiment.get_items()` method is where item recommendations are retrieved for the experiment. This method will select the variation using [Thompson Sampling](https://en.wikipedia.org/wiki/Thompson_sampling) as a [Beta Bernoulli sampler](https://en.wikipedia.org/wiki/Bernoulli_distribution). Thompson Sampling is just one of many possible multi-armed bandit algorthims. Two other common algorithms are Eplsilon Greedy and Upper Confidence Bound 1 (UCB-1).

Thompson Sampling can yield more balanced results in marginal cases. A probability (beta) distribution is maintained for each variation based on the conversion rate observed from user behavior. For each exposure, we sample one possible conversion rate from each variation's beta distribution and select the variation the highest conversion rate. The more data that is gathered, the more confident the algorithm becomes.

```python
# from src/recommendations/src/recommendations-service/experimentation/experiment_mab.py

class MultiArmedBanditExperiment(Experiment):
    """ Implementation of the multi-armed bandit problem using the Thompson Sampling approach 
    to exploring variations to identify and exploit the best performing variation
    """
    def __init__(self, table, **data):
        super(MultiArmedBanditExperiment, self).__init__(table, **data)

    def get_items(self, user_id, current_item_id = None, item_list = None, num_results = 10, tracker = None):
        ...
        
        # Determine the variation to use.
        variation_idx = self._select_variation_index()

        # Increment exposure count for variation
        self._increment_exposure_count(variation_idx)

        # Fetch recommendations using the variation's resolver
        variation = self.variations[variation_idx]

        resolve_params = {
            'user_id': user_id,
            'product_id': current_item_id,
            'product_list': item_list,
            'num_results': num_results
        }
        items = variation.resolver.get_items(**resolve_params)

        # Inject experiment details into recommended items list
        rank = 1
        for item in items:
            correlation_id = self._create_correlation_id(user_id, variation_idx, rank)

            item_experiment = {
                'id': self.id,
                'feature': self.feature,
                'name': self.name,
                'type': self.type,
                'variationIndex': variation_idx,
                'resultRank': rank,
                'correlationId': correlation_id
            }

            item.update({ 
                'experiment': item_experiment
            })

            rank += 1

        ...

        return items

    def _select_variation_index(self):
        """ Selects the variation using Thompson Sampling """
        variation_count = len(self.variations)
        exposures = np.zeros(variation_count)
        conversions = np.zeros(variation_count)

        for i in range(variation_count):
            variation = self.variations[i]
            exposures[i] = int(variation.config.get('exposures', 0))
            conversions[i] = int(variation.config.get('conversions', 0))

        # Sample from posterior (this is the Thompson Sampling approach)
        # This leads to more exploration because variations with > uncertainty can then be selected
        theta = np.random.beta(conversions + 1, exposures + 1)

        # Select variation index with highest posterior p of converting
        return np.argmax(theta)
```

### Setup - Import Dependencies

Througout this workshop we will need access to some common libraries and clients for connecting to AWS services. Let's set those up now.

In [ ]:
import boto3
import json
import uuid
import numpy as np
import requests
import pandas as pd
import random
import scipy.stats as scs
import time
import decimal
import seaborn as sns
from scipy.stats import beta
import matplotlib.pyplot as plt

from boto3.dynamodb.conditions import Key
from random import randint

%matplotlib inline
plt.style.use('ggplot')

cmap = plt.get_cmap("tab10", 3)
sns.set_style("whitegrid")

# We will be using a DynamoDB table to store configuration info for our experiments.
dynamodb = boto3.resource('dynamodb')

# Service discovery will allow us to dynamically discover Retail Demo Store resources
servicediscovery = boto3.client('servicediscovery')
# Retail Demo Store config parameters are stored in SSM
ssm = boto3.client('ssm')

# Utility class to convert types for printing as JSON.
class CompatEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, decimal.Decimal):
            if obj % 1 > 0:
                return float(obj)
            else:
                return int(obj)
        else:
            return super(CompatEncoder, self).default(obj)

### Experiment Strategy Datastore

Let's create an experiment using the multi-armed bandit technique.

A DynamoDB table was created by the Retail Demo Store CloudFormation template that we will use to store the configuration information for our experiments. The table name can be found in a system parameter.

In [ ]:
response = ssm.get_parameter(Name='retaildemostore-experiment-strategy-table-name')

table_name = response['Parameter']['Value']     # Do Not Change
print('Experiments DDB table: ' + table_name)
table = dynamodb.Table(table_name)

Next we need to lookup the Amazon Personalize campaign ARN for product recommendations. This is the campaign that was created in the Personalization workshop.

In [ ]:
response = ssm.get_parameter(Name = 'retaildemostore-related-products-campaign-arn')

campaign_arn = response['Parameter']['Value']   # Do Not Change
print('Personalize product recommendations ARN: ' + campaign_arn)

### Create Multi-Armed Bandit Experiment

The Retail Demo Store supports running multiple experiments concurrently. For this workshop we will create a single multi-armed bandit test/experiment that will expose users of a single group to the variation selected by the **MultiArmedBanditExperiment** class. The Recommendations service already has logic that supports this experiment type when an active experiment is detected.

Experiment configurations are stored in a DynamoDB table where each item in the table represents an experiment and has the following fields.

- **id** - Uniquely identified this experience (UUID).
- **feature** - Identifies the Retail Demo Store feature where the experiment should be applied. The name for the product detail related products feature is `product_detail_related`.
- **name** - The name of the experiment. Keep the name short but descriptive. It will be used in the UI for demo purposes and when logging events for experiment result tracking.
- **status** - The status of the experiment (`ACTIVE`, `EXPIRED`, or `PENDING`).
- **type** - The type of test (`ab` for an A/B test, `interleaving` for interleaved recommendations, or `mab` for multi-armed bandit test)
- **variations** - List of configurations representing variations applicable for the experiment. For this experiment, we will configure three variations.

In [ ]:
feature = 'product_detail_related'
experiment_name = 'product_detail_related_mab'

# First, make sure there are no other active experiments so we can isolate
# this experiment for the exercise.
response = table.scan(
    ProjectionExpression='#k', 
    ExpressionAttributeNames={'#k' : 'id'},
    FilterExpression=Key('status').eq('ACTIVE')
)
for item in response['Items']:
    response = table.update_item(
        Key=item,
        UpdateExpression='SET #s = :inactive',
        ExpressionAttributeNames={
            '#s' : 'status'
        },
        ExpressionAttributeValues={
            ':inactive' : 'INACTIVE'
        }
    )

# Query the experiment strategy table to see if our experiment already exists
response = table.query(
    IndexName='feature-name-index',
    KeyConditionExpression=Key('feature').eq(feature) & Key('name').eq(experiment_name),
    FilterExpression=Key('status').eq('ACTIVE')
)

if response.get('Items') and len(response.get('Items')) > 0:
    print('Experiment already exists')
    product_detail_experiment = response['Items'][0]
else:
    print('Creating experiment')
    
    # Default product resolver
    variation_0 = {
        'type': 'product'
    }
    
    # Similar products resolver
    variation_1 = {
        'type': 'similar'
    }
    
    # Amazon Personalize resolver
    variation_2 = {
        'type': 'personalize-recommendations',
        'campaign_arn': campaign_arn
    }

    product_detail_experiment = { 
        'id': uuid.uuid4().hex,
        'feature': feature,
        'name': experiment_name,
        'status': 'ACTIVE',
        'type': 'mab',
        'variations': [ variation_0, variation_1, variation_2 ]
    }
    
    response = table.put_item(
       Item=product_detail_experiment
    )

    print(json.dumps(response, indent=4))
    
print('Experiment item:')
print(json.dumps(product_detail_experiment, indent=4, cls=CompatEncoder))

## Load Users

For our experiment simulation, we will load all Retail Demo Store users and run the experiment until the sample size has been met.

First, let's discover the IP address for the Retail Demo Store's Users service.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='users',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

users_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Users Service Instance IP: {}'.format(users_service_instance))

Next, let's load all users into a local data frame.

In [ ]:
# Load all 5K users so we have enough to satisfy our sample size requirements.
response = requests.get('http://{}/users/all?count=5000'.format(users_service_instance))
users = response.json()
users_df = pd.DataFrame(users)
pd.set_option('display.max_rows', 5)

users_df

## Load Products

Next let's load products from the Products microservice so we can represent a "current product" to the Recommendations service.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='products',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

products_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Products Service Instance IP: {}'.format(products_service_instance))

In [ ]:
# Load all products.
response = requests.get('http://{}/products/all'.format(products_service_instance))
products = response.json()
products_df = pd.DataFrame(products)
pd.set_option('display.max_rows', 5)

products_df

## Discover Recommendations Service

Next, let's discover the IP address for the Retail Demo Store's Recommendation service.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='recommendations',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

recommendations_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Recommendation Service Instance IP: {}'.format(recommendations_service_instance))

## Simulate Experiment

Next we will simulate our multi-armed bandit experiment by making calls to the Recommendation service across the users we just loaded.

### Simulation Function

The following `simulate_experiment` function is supplied with the number of trials we want to run and the probability of conversion for each variation for our simulation. It runs the simulation long enough to satisfy the number of trials and calls the Recommendations service for each trial in the experiment.

In [ ]:
def simulate_experiment(n_trials, probs):
    """Simulates experiment based on pre-determined probabilities

    Example:

    Parameters:
        n_trials (int): number of trials to perform
        probs (array float): array of floats containing probability/conversion 
            rate for each variation

    Returns:
        df (df) - data frame of simulation data/results
    """

    # will hold exposure/outcome data
    data = []

    print('Simulating experiment for {} users... this may take a few minutes'.format(n_trials))

    for idx in range(n_trials):
        if idx > 0 and idx % 500 == 0:
            print('Simulated experiment for {} users so far'.format(idx))
            
        row = {}

        # Get random user
        user = users[randint(0, len(users)-1)]

        # Get random product
        product = products[randint(0, len(products)-1)]

        # Call Recommendations web service to get recommendations for the user
        response = requests.get('http://{}/recommendations?userID={}&currentItemID={}&feature={}'.format(recommendations_service_instance, user['id'], product['id'], feature))

        recommendations = response.json()
        recommendation = recommendations[randint(0, len(recommendations)-1)]
        
        variation = recommendation['experiment']['variationIndex']
        exposures[variation] += 1
        row['variation'] = variation
        
        # Conversion based on probability of variation
        row['converted'] = np.random.binomial(1, p=probs[variation])

        if row['converted'] == 1:
            # Update experiment with outcome/conversion
            correlation_id = recommendation['experiment']['correlationId']
            requests.post('http://{}/experiment/outcome'.format(recommendations_service_instance), data={'correlationId':correlation_id})
            conversions[variation] += 1
            
        data.append(row)
        
        theta = np.random.beta(conversions + 1, exposures + 1)
        thetas[idx] = theta[variation]
        thetaregret[idx] = np.max(thetas) - theta[variation]

        ad_i[idx] = variation
        
    # convert data into pandas dataframe
    df = pd.DataFrame(data)
    
    print('Done')

    return df

### Run Simulation

Next we run the simulation by defining our simulation parameters for the number of trials and probabilities and then call `simulate_experiment`. This will take a few minutes to run.

In [ ]:
%%time

# Number of users/trials
N = 2000
# Probabilities/payouts for variations
probs = [ 0.08, 0.09, 0.15 ]

# Structures used for experiment analysis
exposures = np.zeros(len(probs))
conversions = np.zeros(len(probs))
theta = np.zeros(len(probs))
thetas = np.zeros(N)
thetaregret = np.zeros(N)
ad_i = np.zeros(N)

# Run the simulation
exp_data = simulate_experiment(N, probs)

In [ ]:
# Display some of the data
exp_data

### Inspect Experiment Summary Statistics

Since the **Experiment** class updates statistics on the experiment in the experiment strategy table when a user is exposed to an experiment ("exposure") and when a user converts ("outcome"), we should see updated counts on our experiment.

In [ ]:
response = table.get_item(Key={'id': product_detail_experiment['id']})

print(json.dumps(response['Item'], indent=4, cls=CompatEncoder))

Note the `conversions` and `exposures` counts for each variation above. These counts were incremented by the experiment class each time a trial was run (exposure) and a user converted in the `simulate_experiment` function above.

### Analyze Simulation Results

To wrap up, let's analyze some of the results from our simulated A/B test by inspecting the actual conversion rate and verifying our target confidence interval and power.

First, let's summarize the results.

In [ ]:
exp_summary = exp_data.pivot_table(values='converted', index='variation', aggfunc=np.sum)
# add additional columns to the pivot table
exp_summary['total'] = exp_data.pivot_table(values='converted', index='variation', aggfunc=lambda x: len(x))
exp_summary['rate'] = exp_data.pivot_table(values='converted', index='variation')

In [ ]:
exp_summary

#### Plot Variation Selections

Let's take a closer look at how our experiment optimized for the best performing variation (2) yet continued to explore variations 0 and 1 through the experiment.

In [ ]:
plt.figure(figsize=(20,5))
x = np.arange(0, N)
plt.scatter(x, ad_i, cmap=cmap, c=ad_i, marker=".", alpha=1)
plt.title("Thompson Sampler - variation selections")
plt.xlabel("Trial")
plt.ylabel("Variation")
plt.yticks(list(range(len(probs))))
cbar = plt.colorbar()
cbar.ax.locator_params(nbins=len(probs))

#### Regret

An additional means of assessing the algorithm's performance is through the concept of regret. Intuitively, regret is quite simple. The algorithm’s regret concerning its action (what variation to show) should be as low as possible. Simply, regret is the difference between the best performance from a variation so far and the performance from the variation chosen for the current trial t.

In [ ]:
best_arm = exp_data.groupby('variation')['converted'].mean().idxmax()
best_value = exp_data.groupby('variation')['converted'].mean().max()
theregret = np.cumsum(best_value - exp_data.converted)
worstregret = np.cumsum(best_value - exp_data.converted*0)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(theregret / (1+np.arange(len(theregret))), label='true regret')
plt.plot(worstregret / (1+np.arange(len(worstregret))), '--', label='avoid linear regret')
plt.ylim(best_value*-0.2, best_value*1.2)
plt.legend()
plt.xlabel("Trial #")
plt.ylabel("Regret")
plt.title("Thompson Sampler regret")

Another nice property of the Thompson algorithm, is that its Bayesian properties mean that we can fully inspect the uncertainty of its payout rate. Let's plot the posterior distributions. You can see how the distributions gradually begin to converge towards the variation with the best payout rate.

In [ ]:
plt.figure(figsize=(16,4))
cmapi = iter(plt.cm.tab10(list(range(len(probs)))))
x = np.arange(0, max(theta) + 0.2, 0.0001)
for i in range(len(probs)):
    pdf = beta(conversions[i], exposures[i]).pdf(x)
    c = next(cmapi)
    plt.plot(x, pdf, c=c, label='variation {}'.format(i), linewidth=3, alpha=.6)
plt.title('Beta distributions after {} trials'.format(N))
plt.legend();

## Conclusion

You have completed all three exercises for the Retail Demo Store Experimentation workshop. Although we focused on experimenation around different approaches to personalization, these experimentation techniques can be applied to many other user experiences in your website. 

We started with a traditional A/B test where a default product recommendation approach was tested against personalized product recommendations from Amazon Personalize. Then we used an interleaved experiment to test two product recommendation approaches concurrently to shorten the testing duration. Finally, we deployed a multi-armed bandit approach to test 3 personalization approaches to maximize exploitation of the best performing variation while still exploring across all other variations. It's important to note that these techniques are not mutually exlusive. In other words, it's common to use interleaving or multi-armed bandit experiments as a preliminary step to identify the best performing variations from a larger pool followed by A/B experiments of the top performers.

### References and Further Reading

- [Multi-armed Bandit Problem](https://en.wikipedia.org/wiki/Multi-armed_bandit), Wikipedia
- [Thompson sampling](https://en.wikipedia.org/wiki/Thompson_sampling), Wikipedia
- [Beta distribtion](https://en.wikipedia.org/wiki/Beta_distribution), Wikipedia
- [Understanding the beta distribution](http://varianceexplained.org/statistics/beta_distribution_and_baseball/), David Robinson
- [Solving multiarmed bandits: A comparison of epsilon-greedy and Thompson sampling](https://towardsdatascience.com/solving-multiarmed-bandits-a-comparison-of-epsilon-greedy-and-thompson-sampling-d97167ca9a50), Conor McDonald